<h2>Segmenting and Clustering Neighbourhoods in Toronto</h2>

In this assignment, you will be required to explore, segment, and cluster the neighborhoods in the city of Toronto. 
For this assignment, you will be required to explore and cluster the neighborhoods in Toronto.
1. Start by creating a new Notebook for this assignment.
2. Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe.
3. To create the above dataframe:
The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.
If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.
4. Submit a link to your Notebook on your Github repository. (10 marks)

<h3>All the 3 tasks of <i>web scraping</i>, <i>cleaning</i> and <i>clustering</i> are implemented in the same notebook for the ease of evaluation.</h3>

<h3>Installing and Importing the required Libraries</h3>

In [ ]:
#install the BeautifulSoup package for web scraping
!pip install beautifulsoup4
from bs4 import BeautifulSoup

#install lxml for processing XML and HTML
!pip install lxml

#import library for requests handling
import requests as rq
#import library for data analsysis
import pandas as pd
#import library for vectorized data handling
import numpy as np
#import library for random number generation
import random 

#import geopy module to convert address info to latitude and longitude
from geopy.geocoders import Nominatim 

#import libraries for displaying images and html
from IPython.display import Image 
from IPython.core.display import HTML 
from IPython.display import display_html
    
#import library for tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

#install folium 
!conda install -c conda-forge folium=0.5.0 --yes

#import plotting library
import folium 

from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

print('Folium installed')
print('Libraries imported.')

<h3>Task 1 Scraping the Wikipedia page for the table of postal codes of Canada and create a dataframe</h3>

BeautifulSoup Library of Python is used for scraping of data table from the Wikipedia page, and title of the webpage is printed to checkas well as head of the table of postal codes of Canada is printed.

In [ ]:
#scrapping the wiki page from https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
wiki_source = rq.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
wiki_soup = BeautifulSoup(wiki_source,'lxml')
print(wiki_soup.title)

<h3>Cover html table to Pandas DataFrame for cleaning and preprocessing.</h3>

In [ ]:
#retrieve table
wiki_tab = str(wiki_soup.table)
dfs = pd.read_html(wiki_tab)

#convert table to dataframe
df=dfs[0]
df.head()

<h3>Table data preprocessing and cleaning</h3>

In [ ]:
# Drop rows where Borough is 'Not assigned' as required to oinly process the cells that have an assigned borough and ignore cells with a borough that is Not assigned.
df_noborough = df[df.Borough != 'Not assigned']

# Merge neighbourhoods with same Postalcode
df_merged = df_noborough.groupby(['Postal Code','Borough'], sort=False).agg(', '.join)
df_merged.reset_index(inplace=True)

# Replacing the name of the neighbourhoods which are 'Not assigned' with names of Borough
df_merged['Neighbourhood'] = np.where(df_merged['Neighbourhood'] == 'Not assigned',df_merged['Borough'], df_merged['Neighbourhood'])
print(df_merged.head(12))

In [ ]:
# Shape of data frame
df_merged.shape

<h3>Importing the csv file conatining the latitudes and longitudes for various neighbourhoods in Canada</h3>

In [ ]:
lat_lon = pd.read_csv('https://cocl.us/Geospatial_data')
lat_lon.head()

<h3>Merging the two tables for getting the Latitudes and Longitudes for various neighbourhoods in Canada</h3>

In [ ]:
lat_lon.rename(columns={'Postal Code':'Postcode'},inplace=True)
df_lat_lon = pd.merge(df_merged,lat_lon,on='Postcode')
df_lat_lon.head()

<h2>The notebook from here includes the Clustering and the plotting of the neighbourhoods of Canada which contain Toronto in their Borough</h2>

<h3>Getting all the rows from the data frame which contains Toronto in their Borough.</h3>

In [ ]:
df4 = df3[df3['Borough'].str.contains('Toronto',regex=False)]
df4

<h3>Visualizing all the Neighbourhoods of the above data frame using Folium</h3>

In [ ]:
map_toronto = folium.Map(location=[43.651070,-79.347015],zoom_start=10)

for lat,lng,borough,neighbourhood in zip(df4['Latitude'],df4['Longitude'],df4['Borough'],df4['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
    [lat,lng],
    radius=5,
    popup=label,
    color='blue',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_toronto)
map_toronto

<h3>The map might not be visible on Github. Check out the README for the map.</h3>

<h3>Using KMeans clustering for the clsutering of the neighbourhoods</h3>

In [ ]:
k=5
toronto_clustering = df4.drop(['Postcode','Borough','Neighbourhood'],1)
kmeans = KMeans(n_clusters = k,random_state=0).fit(toronto_clustering)
kmeans.labels_
df4.insert(0, 'Cluster Labels', kmeans.labels_)


In [ ]:
df4

In [ ]:
# create map
map_clusters = folium.Map(location=[43.651070,-79.347015],zoom_start=10)

# set color scheme for the clusters
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, neighbourhood, cluster in zip(df4['Latitude'], df4['Longitude'], df4['Neighbourhood'], df4['Cluster Labels']):
    label = folium.Popup(' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<h3>The map might not be visible on Github. Check out the README for the map.</h3>